![CC-BY-SA](https://mirrors.creativecommons.org/presskit/buttons/88x31/svg/by-sa.svg)


This notebook was created by [Bernardo Freitas Paulo da Costa](http://www.im.ufrj.br/bernardofpc),
and is licensed under Creative Commons BY-SA.

Antes de enviar este Teste, verifique que tudo está funcionando como esperado.
Por exemplo, **rode o código inteiro, do zero**.
Para isso, vá no menu, escolha _Kernel_, depois _Restart & Run All_.

Verifique, também, que você respondeu todas as questões:
* as questões de código têm `YOUR CODE HERE` (e você pode apagar o `raise NotImplemented` ao incluir sua resposta)
* as questões discursivas têm "YOUR ANSWER HERE".

---

In [1]:
import numpy as np

# Teste 1: Bisseção

## Questão 1: Número de iterações da bisseção, e de chamadas à função `f`

Generalize o algoritmo da bisseção _iterativa_ para retornar,
- a (aproximação da) raiz,
- o número de bisseções (divisões por 2 do intervalo) feitas,
- o número de vezes que você chamou a função `f`

Use como critério de parada
- `xtol`, e
- `maxiter`,

ou seja, o algoritmo termina quando:
- seja possível garantir que o erro (absoluto) da resposta ("em $x$") seja menor do que `xtol`; OU
- quando o algoritmo já tiver feito `maxiter` itererações.

## Validação da função bissecao

A cédula abaixo importa e define tipos e funções utilizados na função de validação da bisseção e na na própria função que implementa o algoritimo da bisseção.

In [2]:
from typing import List, Union

FloatInt = Union[float, int]

def isFloatInt(n: any) -> bool:
    '''Verifica se n é um float ou inteiro'''
    return type(n) in [float, int] or np.isreal(n)

def all_have_same_sign(l: List[FloatInt]) -> bool:
    '''Verifica se todos os elementos de uma lista tem o mesmo sinal'''
    return not min(l) < 0 < max(l)

Definimos então a função que valida os argumentos passados para a função bissecao:

In [3]:
from inspect import isfunction
from typing import Callable

def validate_bissect_args(f: Callable, a: FloatInt, b: FloatInt, xtol: FloatInt = 1e-8, maxiter: int = 10) -> None:
    '''Verifica se os argumentos passados para a função bisseção estão corretos, i.e,
    se não podem causar um erro inesperado quando a função for invocada'''
    assert isfunction(f), f'O parâmetro f: {f} deve ser uma função'
    assert isFloatInt(a), f'O parâmetro a: {a} deve ser um float ou um inteiro'
    assert isFloatInt(b), f'O parâmetro b: {b} deve ser um float ou um inteiro'
    assert a != b, f'Os números a: {a} e b: {b} devem ser distintos'
    assert not all_have_same_sign(
        [f(a), f(b)]), f'Os valores de f({a}) = {f(a)} e f({b}) = {f(b)} devem ter sinais opostos'
    assert isFloatInt(xtol), f'O parâmetro xtol: {xtol} deve ser um float ou um inteiro'
    assert xtol > 0, f'O número xtol: {xtol} deve ser positivo'
    assert type(
        maxiter) == int, f'O parâmetro maxiter {maxiter} deve ser um inteiro'
    assert maxiter > 0, f'O número de iterações {maxiter} deve ser positivo'

Finalmente, definimos a função que implementa o algoritimo da bisseção para obter uma raíz de uma função dentro de certas restrições:

In [4]:
def bissecao(f: Callable, a: FloatInt, b: FloatInt, xtol: FloatInt = 1e-8, maxiter: int = 10):
    '''Encontra uma raíz de f pertencente ao intervalo [a,b]'''
    validate_bissect_args(f, a, b, xtol, maxiter)

    niters: int = 0
    low: FloatInt = min([a, b])
    high: FloatInt = max([a, b])

    def get_mid(high, low) -> FloatInt:
        return (high + low) / 2

    mid = get_mid(high, low)
    while abs(high - low) > xtol and niters < maxiter:
        niters += 1
        if f(mid) > 0:
            high = mid
        else:
            low = mid
        mid = get_mid(high, low)

    root = mid
    ncalls = niters
    return (root, niters, ncalls)

### 1.1: Testando com $\sqrt2$:

In [5]:
def p2(x): return x**2 - 2

In [6]:
raiz, niters, ncalls = bissecao(p2, 0, 2, maxiter=100)
assert abs(raiz - np.sqrt(2)) < 1e-8
assert 25 <= niters <= 30
assert 25 <= ncalls

In [7]:
raiz, niters, ncalls = bissecao(p2, 0, 2, xtol=1e-3)
assert abs(raiz - np.sqrt(2)) < 1e-3
assert 9 <= niters <= 11

In [8]:
raiz, niters, ncalls = bissecao(p2, 0, 2)
assert abs(raiz**2 - 2) > 1e-3
assert niters == 10

### 1.2: Tolerância, número de iterações, respostas, ...

Observe o seguinte código:

In [9]:
for err in np.logspace(-8, 0, num=9):
    r, ni, nc = bissecao(p2, 0, 2, xtol=err)
    print(f"xtol = {err:.1e} --> raiz = {r:.10f}, #iter = {ni:3d}")

xtol = 1.0e-08 --> raiz = 1.4150390625, #iter =  10
xtol = 1.0e-07 --> raiz = 1.4150390625, #iter =  10
xtol = 1.0e-06 --> raiz = 1.4150390625, #iter =  10
xtol = 1.0e-05 --> raiz = 1.4150390625, #iter =  10
xtol = 1.0e-04 --> raiz = 1.4150390625, #iter =  10
xtol = 1.0e-03 --> raiz = 1.4150390625, #iter =  10
xtol = 1.0e-02 --> raiz = 1.4179687500, #iter =   8
xtol = 1.0e-01 --> raiz = 1.4062500000, #iter =   5
xtol = 1.0e+00 --> raiz = 1.5000000000, #iter =   1


Comente

Para os primeiros valores de xtol, à medida que dimínuimos seu valor, aumentamos o número de iterações necessárias para obter o valor de uma raíz com a precisão desejada. Mas quando xtol atinge 1.0e-03, sua diminuição (mesmo que em ordem 10) não implica numa raíz mais precisa, porque o número de iterações (cujo valor máximo default é 10) se manifesta como critério de parada e impede a continuação do algoritimo.

Como seria o equivalente se o critério fosse o número de iterações?

In [10]:
for miter in list(range(100,0,-10)):
    r, ni, nc = bissecao(p2, 0, 2, maxiter=miter)
    print(f"maxiter = {miter} --> raiz = {r:.20f}, #iter = {ni:3d}")

maxiter = 100 --> raiz = 1.41421356424689292908, #iter =  28
maxiter = 90 --> raiz = 1.41421356424689292908, #iter =  28
maxiter = 80 --> raiz = 1.41421356424689292908, #iter =  28
maxiter = 70 --> raiz = 1.41421356424689292908, #iter =  28
maxiter = 60 --> raiz = 1.41421356424689292908, #iter =  28
maxiter = 50 --> raiz = 1.41421356424689292908, #iter =  28
maxiter = 40 --> raiz = 1.41421356424689292908, #iter =  28
maxiter = 30 --> raiz = 1.41421356424689292908, #iter =  28
maxiter = 20 --> raiz = 1.41421413421630859375, #iter =  20
maxiter = 10 --> raiz = 1.41503906250000000000, #iter =  10


Comente, também.

Agora, para valores baixos do número máximo de iterações (maxiter), seu aumento é acompanhado por um aumento no número de iterações do programa e na precisão do valor da raíz. Mas quando maxiter atinge 30, notamos que o algoritimo sempre executa no máximo 28 iterações - porque o outro critério de parada, de tolerância de xtol (cujo valor default é 1e-8), é atingido.

### 1.3: Agora, um polinômio um pouco mais complicado:

In [11]:
def p4(x): return x**4 - 3*x + 1

In [12]:
r1, n1, c1 = bissecao(p4, 0, 1, maxiter=100)
assert p4(r1) < 1e-6

In [13]:
r2, n2, c2 = bissecao(p4, 1, 2, maxiter=100)
assert n1 == n2
assert p4(r2) < 1e-6

In [14]:
r3, n3, c2 = bissecao(p4, 1, 3, maxiter=100)
assert n3 == n2+1
assert r3 == r2

### 1.4: Relação entre `ncalls` e `niters`

Muito provavelmente, é possível calcular `ncalls` a partir de `niters`.
Qual a relação, para o seu programa, entre estes valores?
Porquê?

In [15]:
# Use esta caixa para evidenciar a relação, e a de baixo para explicar qual é, e porquê
# YOUR CODE HERE
# MUDAR PARA raise NotImplementedError() no final, uma vez que não obtive uma respota que possa ser mostrada em código nessa caixa
raise NotImplementedError()

NotImplementedError: 

Confesso que essa questão ficou um pouco confusa pra mim - definitivamente por falta de entendimento do problema.

Inicialmente, supus que o valor de niters representa o número de bisseções (divisões) do intervalo em x (onde a raíz se encontra) realizadas pelo programa. Mas se essa suposição for verdadeira, o algoritimo implementado não passa no teste "assert niters == 10" da questão 1.2, uma vez que uma bisseção deve ser realizada antes do loop (while ...) ser executado e o número de iterações passa a ser 11. Então assumi que niters é o número de iterações dentro do loop realizadas pelo programa. 

Partindo dessa premissa, e considerando que o valor de ncalls corresponde ao número de chamadas à função f realizadas - que, no algoritimo implementado, devem ser realizadas para determinar se o intervalo em x onde se encontra a raíz produz valores super ou subestimados - então sempre que uma iteração do loop é executada, ambos os valores de niters e ncalls são aumentados em uma unidade e portanto ncalls == niters, relação evidenciada no código contido na linha "ncalls = niters".


## Questão 2: Bisseção em listas ordenadas

Se desejamos encontrar um número em uma lista, podemos percorrer cada um de seus índices
até achar o número, e retornamos o índice correspondente.

Mas, se a lista `l` possui números em ordem,
é possível usar uma variante da bisseção para encontrar este índice.
Observe que agora desejamos retornar um número **inteiro** (o índice!),
e que índices para listas sempre devem ser inteiros, portanto:
- cuidado ao escolher o "ponto médio"
- mas o critério de parada é "quando achar o índice".

### 2.1: Encontrando um número

Implemente a função `bissect_list(l, v)`, que retorna
- o índice `k` do elemento na lista `l`, que vale `v`; e
- o número de acessos à lista `l`.

Suponha que a lista `l` é não-decrescente (ou seja, `l[i] <= l[i+1]`).

In [17]:
from typing import Tuple

def bissect_list(l: List[any], v: any) -> Tuple[None or int, int]:
    """Índice  k  na lista  l , crescente, tal que l[k] = v.  None caso não exista."""
    low: int = 0
    high: int = len(l)
    naccess: int = 0

    while high - low >= 1:
        naccess += 1
        k = (high + low) // 2
        if high - low == 1:
            return (k if l[k] == v else None, naccess)
        elif l[k] == v:
            return (k, naccess)
        elif l[k] > v:
            high = k
        else:
            low = k
    return (None, 0)

In [18]:
assert bissect_list([], 42) == (None,0)

In [19]:
l = [4]
assert bissect_list(l, 4) == (0,1)

In [20]:
assert bissect_list(l, 5) == (None,1)

In [21]:
l = [1,3,6,10,15,21,28]
for i,v in enumerate(l):
    idx, n = bissect_list(l, v)
    assert idx == i, f"v={v}, i={i}, idx={idx}"
    print(n)

3
2
3
1
3
2
3


Qual o maior número de acessos à lista que você precisou para encontrar um valor nela?
Este número depende da posição na lista?  Explique!

3. Sim, depende da posição do elemento buscado na lista, uma vez que o algoritimo implementado sempre inicia buscando valores próximos da mediana do index da lista - e portanto se o valor buscado existir e calhar de ter um index igual ao primeiro index buscado, ele será encontrado com apenas um acesso à lista.

### 2.2: Uma lista muito maior

In [22]:
l = np.arange(-4000, 6000)
np.random.seed(1)
valores_testes = np.random.randint(low=-4000, high=6000, size=10)
for v in valores_testes:
    idx, n = bissect_list(l, v)
    assert l[idx] == v
    assert n <= 20
    print(f"v={v:5d}, idx={idx:5d}, n={n:3d}")

v=-3765, idx=  235, n= 13
v= 1192, idx= 5192, n= 12
v=-3095, idx=  905, n= 13
v= 3813, idx= 7813, n= 13
v=-1105, idx= 2895, n= 13
v= 1056, idx= 5056, n= 13
v=-3856, idx=  144, n= 13
v=  225, idx= 4225, n= 13
v= 3751, idx= 7751, n= 13
v= -538, idx= 3462, n= 13


Como o número de acessos varia, nesta outra lista?
E como ele se compara com o tamanho da lista?

Ele permanece quase constante para as buscas realizadas - o que indica que provavelmente o desvio-padrão do número de acessos em uma lista maior é menor.

Como o algoritimo implementado funciona divindo a lista onde o elemento é buscado em 2 para cada iteração (e consequente acesso à lista), depreende-se que o o valor máximo de acessos para uma determina lista com tamanho N é log(N) na base 2 (já que log(N) na base 2 é o número de vezes que 2 deve ser multiplicado por si mesmo para chegar a N). Portanto o número de acessos depende do tamanho da lista.

### 3.3: Listas crescentes e decrescentes.

Altere o código anterior, para receber um argumento a mais, `decr`, que indica que a lista é **decrescente**.
Não reordene a lista!

In [23]:
def bissect(l, v, decr=False):
    """Index  k  on a monotonic list  l  such that  l[k] == v.
    Returns  None  if  not present.
    If  l  is decresing, pass  decr=True."""
    low: int = 0
    high: int = len(l)
    naccess: int = 0

    while high - low >= 1:
        naccess += 1
        k = (high + low) // 2
        comparison = l[k] < v if decr else l[k] > v

        if high - low == 1:
            return (k if l[k] == v else None, naccess)
        elif l[k] == v:
            return (k, naccess)
        elif comparison:
            high = k
        else:
            low = k
    return (None, 0)

In [24]:
l = [1,3,6,10,15,21,28,35,36]
assert bissect(l, 15)[0] == 4

In [25]:
l_rev = [36, 35, 28, 21, 15, 10, 6, 3, 1]
assert bissect(l_rev, 15, decr=True)[0] == 4

In [26]:
l_rev = [36, 35, 28, 21, 15, 10, 6, 3, 1]
assert bissect(l_rev, 16, decr=True)[0] == None

In [27]:
l3 = [36, 35, 28, 21, 15, 10, 9, 6, 3, 2, 1]
assert bissect(l3, 15, decr=True)[0] == 4

### 2.4: Bônus

O número de acessos à lista para encontrar um número muda se esta está invertida?
Explique!

Intuitivamente (rsrs), não. Independente da lista estar ordenada crescente ou decrescentemente, a implementação continua consistindo em dividir a lista em duas partes a cada iteração, portanto não vejo como o número de acessos poderia ser alterado a depender da ordenação.